# Overview of signal processing results for ecg derived respiration

In [120]:
import pandas as pd

## BIDMC
Results from RRest toolbox saved at: bidmc_results

Component data: per subject the CC's are store in separated file *_CC.mat; also each file has CC values per algorithm and per window

### Calculating CC's

In [68]:
import scipy.io
import numpy as np
import os
import glob

def load_mat_files(directory):
    """Load all *_cc.mat files from the given directory."""
    files = glob.glob(os.path.join(directory, '*_cc.mat'))
    data = {}
    for file in files:
        subject_id = os.path.basename(file).split('_')[0]
        mat_data = scipy.io.loadmat(file)
        data[subject_id] = mat_data
    return data

def extract_values(data, signal_prefix='ekg'):
    """Extract CC & MSE values for a specific signal type from the loaded mat files."""
    cc_values = {}
    mse_values = {}
    
    for subject, mat_data in data.items():
        for key in mat_data:
            if key.startswith(signal_prefix) and isinstance(mat_data[key], np.ndarray):
                if 'CCp' in mat_data[key].dtype.names:
                    method = key.split('_', 1)[1]  # Extract method name
                    cc_values.setdefault(subject, {}).setdefault(method, []).extend(mat_data[key]['CCp'][0])
    
                if 'MSE' in mat_data[key].dtype.names:
                        method = key.split('_', 1)[1]  # Extract method name
                        mse_values.setdefault(subject, {}).setdefault(method, []).extend(mat_data[key]['MSE'][0])
    return cc_values, mse_values

def calculate_median(values):
    """Calculate the median for each subject and each method."""
    median_values = {}
    for subject, methods in values.items():
        median_values[subject] = {}
        for method, value in methods.items():
            median_values[subject][method] = np.nanmedian(value)
    return median_values

def calculate_mean(values):
    """Calculate the mean for each subject and each method."""
    mean_values = {}
    for subject, methods in values.items():
        mean_values[subject] = {}
        for method, value in methods.items():
            mean_values[subject][method] = np.nanmean(value)
    return mean_values

def calculate_overall_median(median):
    """Calculate the overall median across all subjects for each method."""
    method_values = {}
    for methods in median.values():
        for method, value in methods.items():
            method_values.setdefault(method, []).append(value)
    for method, values in method_values.items():
        overall_median = {method: np.nanmedian(values) for method, values in method_values.items()}
    return overall_median

def calculate_overall_mean(mean):
    """Calculate the overall mean across all subjects for each method."""
    method_values = {}
    for methods in mean.values():
        for method, value in methods.items():
            method_values.setdefault(method, []).append(value)
    for method, values in method_values.items():
        overall_mean = {method: np.nanmedian(values) for method, values in method_values.items()}
    return overall_mean

In [69]:
directory = './bidmc_results/Analysis_files/Component_Data/'

In [70]:
data = load_mat_files(directory)
cc_values, mse_values = extract_values(data)

In [71]:
median_values_cc = calculate_median(cc_values)
median_values_mse = calculate_median(mse_values)

In [72]:
mean_values_cc = calculate_mean(cc_values)
mean_values_mse = calculate_mean(mse_values)

In [73]:
calculate_overall_median(median_values_cc)

{'ELF_RSlinB_FMeam_FPt_RDtGC_EHF': 0.6689206558933192,
 'ELF_RSlinB_FMebw_FPt_RDtGC_EHF': 0.6244211874382042,
 'ELF_RSlinB_FMefm_FPt_RDtGC_EHF': 0.32408458191380596,
 'flt_BFi': 0.5806797173223289,
 'flt_Wam': 0.36963336938242153,
 'flt_Wfm': 0.29898291638369046}

In [74]:
calculate_overall_mean(mean_values_cc)

{'ELF_RSlinB_FMeam_FPt_RDtGC_EHF': 0.6372758821727452,
 'ELF_RSlinB_FMebw_FPt_RDtGC_EHF': 0.6002211256607378,
 'ELF_RSlinB_FMefm_FPt_RDtGC_EHF': 0.3268251567073985,
 'flt_BFi': 0.5586074061739241,
 'flt_Wam': 0.38144137099571845,
 'flt_Wfm': 0.31143610106791636}

In [77]:
calculate_overall_median(median_values_mse)

{'ELF_RSlinB_FMeam_FPt_RDtGC_EHF': 0.06548734871978001,
 'ELF_RSlinB_FMebw_FPt_RDtGC_EHF': 0.06352932768885228,
 'ELF_RSlinB_FMefm_FPt_RDtGC_EHF': 0.10049166727873382,
 'flt_BFi': 0.07339123243711032,
 'flt_Wam': 0.09864817470949042,
 'flt_Wfm': 0.1083998867323163}

In [78]:
calculate_overall_mean(mean_values_mse)

{'ELF_RSlinB_FMeam_FPt_RDtGC_EHF': 0.0656807723440914,
 'ELF_RSlinB_FMebw_FPt_RDtGC_EHF': 0.06489365945136012,
 'ELF_RSlinB_FMefm_FPt_RDtGC_EHF': 0.10157131195548069,
 'flt_BFi': 0.07674733331598545,
 'flt_Wam': 0.10882017625720494,
 'flt_Wfm': 0.11178725031768236}

## Capnobase

In [51]:
directory = './capnobase_results/Analysis_files/Component_Data/'

In [52]:
data = load_mat_files(directory)
cc_values, mse_values = extract_values(data)

In [53]:
median_values_cc = calculate_median(cc_values)
median_values_mse = calculate_median(mse_values)

In [54]:
mean_values_cc = calculate_mean(cc_values)
mean_values_mse = calculate_mean(mse_values)

In [55]:
calculate_overall_median(median_values_cc)

{'ELF_RSlinB_FMeam_FPt_RDtGC_EHF': 0.8611708902922213,
 'ELF_RSlinB_FMebw_FPt_RDtGC_EHF': 0.6931336921888314,
 'ELF_RSlinB_FMefm_FPt_RDtGC_EHF': 0.6897644243814938,
 'flt_BFi': 0.45598445558912803,
 'flt_Wam': 0.45213411987543184,
 'flt_Wfm': 0.40757009418580525}

In [56]:
calculate_overall_mean(mean_values_cc)

{'ELF_RSlinB_FMeam_FPt_RDtGC_EHF': 0.8330416025511844,
 'ELF_RSlinB_FMebw_FPt_RDtGC_EHF': 0.6596325235201358,
 'ELF_RSlinB_FMefm_FPt_RDtGC_EHF': 0.6599929731951029,
 'flt_BFi': 0.4188092218740107,
 'flt_Wam': 0.42940958605211926,
 'flt_Wfm': 0.4140829981639218}

In [57]:
calculate_overall_median(median_values_mse)

{'ELF_RSlinB_FMeam_FPt_RDtGC_EHF': 0.04839334326332245,
 'ELF_RSlinB_FMebw_FPt_RDtGC_EHF': 0.08120463431092063,
 'ELF_RSlinB_FMefm_FPt_RDtGC_EHF': 0.0759892699823336,
 'flt_BFi': 0.11813423559092215,
 'flt_Wam': 0.12086288733420263,
 'flt_Wfm': 0.12552053905685007}

In [58]:
calculate_overall_mean(mean_values_mse)

{'ELF_RSlinB_FMeam_FPt_RDtGC_EHF': 0.057340347402984417,
 'ELF_RSlinB_FMebw_FPt_RDtGC_EHF': 0.08609671091004677,
 'ELF_RSlinB_FMefm_FPt_RDtGC_EHF': 0.08289334234411427,
 'flt_BFi': 0.11288783251743041,
 'flt_Wam': 0.12566097266572143,
 'flt_Wfm': 0.13239046138302132}